<a href="https://colab.research.google.com/github/jerondeepak/Peer-graded-Assignment-week-1/blob/main/Deepak_Jeron_9_7_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Uninstall mkl for faster neural-network training time
!pip uninstall -y mkl
# Upgrade pip to ensure the latest package versions are available
!pip install -U pip
# Upgrade setuptools to be compatible with namespace packages
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
# Install autogluon (Tutorial based on autogluon==0.1.0)
!pip install autogluon
# Upgrade ipykernel (Necessary for Colab)

!pip install -U ipykernel

In [2]:
#import neccessary modules
import pandas as pd
import numpy as np
import autogluon.core as ag
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split


DataSet = pd.read_csv('csv_nfl_data.csv')
DataSet.head()

with open('csv_nfl_data.csv','r') as f:
    data = f.read().split('\n')
data[1]=data[0]+data[1]
del data[0]
with open('changed_dataset.csv','w') as f:
    for i in data:
        f.write(i+'\n')


dataset = pd.read_csv('changed_dataset.csv')

dataset.head()

dataset.info()

dataset.drop("Date(IST)",inplace=True,axis=1)
dataset.isnull().sum()

null_list = ['FIIB','FIIS','FIIN','DIIB','DIIS','DIIN','August','December']
for i in null_list:
    dataset[i]=dataset[i].fillna(dataset[i].mean())

dataset.isnull().sum()

#with NFH and without NFL dataset 
dataset_NFH = dataset.drop('NFL',axis=1)
#with NFL and without NFH dataset
dataset_NFL = dataset.drop('NFH',axis=1)

def Train_Test_Split(dataset):
    train, test = train_test_split(dataset , train_size=0.7, test_size=0.3)
    return (train,test)

#NFH dataset split into train and test
train_NFH, test_NFH = Train_Test_Split(dataset_NFH)
#NFL dataset split into train and test
train_NFL, test_NFL = Train_Test_Split(dataset_NFL)

train_NFH.to_csv('train_NFH.csv', index=False)
test_NFH.to_csv('test_NFH.csv',index=False)
train_NFL.to_csv('train_NFL.csv',index=False)
test_NFL.to_csv('test_NFL.csv')




#NFH
train_data = TabularDataset('train_NFH.csv')
subsample_size = 250  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0,)
print(train_data.head())

label = 'NFH'

new_data = TabularDataset('test_NFH.csv')
test_data = new_data  # this should be separate data in your applications
y_test = test_data[label]
test_data_nolabel = test_data.drop(columns=[label])  # delete label column
val_data = new_data



import autogluon.core as ag

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'layers': ag.space.Categorical([100], [1000], [200, 100], [300, 200, 100]),  # each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                     # NOTE: comment this line out if you get errors on Mac OSX
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 2*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using Bayesian optimization routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}



'''
predictor = TabularPredictor(label=label, eval_metric='accuracy').fit(
    train_data, tuning_data=val_data, time_limit=time_limit,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)
'''
predictor = TabularPredictor(label=label).fit(train_data)


y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

results = predictor.fit_summary()





#NFL
train_data = TabularDataset('train_NFL.csv')
subsample_size = 250  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0,)
print(train_data.head())

label = 'NFL'

new_data = TabularDataset('test_NFL.csv')
test_data = new_data  # this should be separate data in your applications
y_test = test_data[label]
test_data_nolabel = test_data.drop(columns=[label])  # delete label column
val_data = new_data

'''
predictor = TabularPredictor(label=label, eval_metric='accuracy').fit(
    train_data, tuning_data=val_data, time_limit=time_limit,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)
'''
predictor = TabularPredictor(label=label).fit(train_data)
y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

results = predictor.fit_summary()




Loaded data from: train_NFH.csv | Columns = 20 / 20 | Rows = 250 -> 250
Loaded data from: test_NFH.csv | Columns = 20 / 20 | Rows = 108 -> 108
No path specified. Models will be saved in: "AutogluonModels/ag-20210710_072447/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20210710_072447/"
AutoGluon Version:  0.2.0
Train Data Rows:    250
Train Data Columns: 19
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (15921.6774310452, 8367.91282781187, 12594.31465, 2110.3326)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    10698.81 MB
	Train 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358 entries, 0 to 357
Data columns (total 22 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   nfl_data   358 non-null    int64  
 1   Date(IST)  358 non-null    object 
 2   NFO        358 non-null    float64
 3   NFH        358 non-null    float64
 4   NFL        358 non-null    float64
 5   NFC        358 non-null    float64
 6   FIIB       357 non-null    float64
 7   FIIS       357 non-null    float64
 8   FIIN       357 non-null    float64
 9   DIIB       357 non-null    float64
 10  DIIS       357 non-null    float64
 11  DIIN       357 non-null    float64
 12  August     356 non-null    float64
 13  December   356 non-null    float64
 14  CAD        358 non-null    float64
 15  DAD        358 non-null    float64
 16  DOD        358 non-null    float64
 17  NDAD       358 non-null    float64
 18  Currey     358 non-null    float64
 19  Flow       358 non-null    float64
 20  Shine     

	-436.2821	 = Validation root_mean_squared_error score
	0.0s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: KNeighborsDist ...
	-433.8118	 = Validation root_mean_squared_error score
	0.0s	 = Training runtime
	0.11s	 = Validation runtime
Fitting model: LightGBMXT ...
	-201.3345	 = Validation root_mean_squared_error score
	0.8s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	train_set's rmse: 52.5048	valid_set's rmse: 201.481


	-115.006	 = Validation root_mean_squared_error score
	0.69s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-66.6312	 = Validation root_mean_squared_error score
	0.62s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	-252.2455	 = Validation root_mean_squared_error score
	1.82s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-44.3296	 = Validation root_mean_squared_error score
	0.52s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-12524.1095	 = Validation root_mean_squared_error score
	1.76s	 = Training runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	-104.8678	 = Validation root_mean_squared_error score
	0.31s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetMXNet ...
	-327.7782	 = Validation root_mean_squared_error score
	3.63s	 = Training runtime
	0.48s	 = Validation runtime
Fitting model: LightGBMLarge ...
	-8

...

[1000]	train_set's rmse: 25.6043	valid_set's rmse: 249.21
[2000]	train_set's rmse: 7.15439	valid_set's rmse: 246.483
[3000]	train_set's rmse: 2.00243	valid_set's rmse: 246.043
[4000]	train_set's rmse: 0.566131	valid_set's rmse: 245.98
[5000]	train_set's rmse: 0.164329	valid_set's rmse: 245.91
[6000]	train_set's rmse: 0.0467448	valid_set's rmse: 245.883
[7000]	train_set's rmse: 0.0127849	valid_set's rmse: 245.877
[8000]	train_set's rmse: 0.00337388	valid_set's rmse: 245.876
[9000]	train_set's rmse: 0.000864839	valid_set's rmse: 245.876
[10000]	train_set's rmse: 0.000228513	valid_set's rmse: 245.876


	-245.8757	 = Validation root_mean_squared_error score
	3.59s	 = Training runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-160.27	 = Validation root_mean_squared_error score
	0.61s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	-359.0143	 = Validation root_mean_squared_error score
	2.89s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-141.9021	 = Validation root_mean_squared_error score
	0.52s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-12118.2267	 = Validation root_mean_squared_error score
	1.72s	 = Training runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	-104.7692	 = Validation root_mean_squared_error score
	0.27s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetMXNet ...
	-428.6511	 = Validation root_mean_squared_error score
	3.73s	 = Training runtime
	0.47s	 = Validation runtime
Fitting model: LightGBMLarge ...
	-

Predictions:   [11774.8232421875, 11143.20703125, 11677.5166015625, 14822.4892578125, 15514.4404296875]


Evaluation: root_mean_squared_error on test data: -93.6247317594914
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -93.6247317594914
}


*** Summary of fit() ***
Estimated performance of each model:
                  model     score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2    -99.523508       0.135426  4.785924                0.000525           0.403340            2       True         12
1               XGBoost   -104.769241       0.002960  0.267415                0.002960           0.267415            1       True          9
2         ExtraTreesMSE   -141.902103       0.102457  0.523154                0.102457           0.523154            1       True          7
3       RandomForestMSE   -160.270044       0.102515  0.611927                0.102515           0.611927            1       True          5
4         LightGBMLarge   -180.243689       0.003457  0.738209                0.003457           0.738209            1       True         11
5              LightGBM   -245.875651       0.029485  3.592015                0.029485      